In [13]:
import torch
import numpy as np
from utils.dataset import SimpleIterDataset
import os
import onnxruntime as ort
import torch.autograd.profiler as profiler

In [14]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [15]:
def create_ort_inputs(inputs, input_names):
    return {name: to_numpy(inputs[index]) for (index, name) in enumerate(input_names)}

In [16]:
def run_ort_inference(onnx_path, inputs, input_names):
    ort_session = ort.InferenceSession(onnx_path)
    runSessionParams = ort.RunOptions()
    runSessionParams.log_severity_level = 2
    ort_out = ort_session.run(None, create_ort_inputs(inputs, input_names), run_options = runSessionParams)
    return ort_out

In [17]:
def run_pt_inference(model, inputs):
    with profiler.profile(with_stack=True, profile_memory=True) as prof:
        output = model(*inputs)
    print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))
    return output

In [18]:
def export_onnx(model, inputs, dynamic_axes, input_names, output_names, onnx_path, opset_version = 15):
    torch.onnx.export(model, inputs, onnx_path, input_names=input_names, output_names=output_names, dynamic_axes=dynamic_axes, opset_version=opset_version)
    model_output = run_pt_inference(model, inputs)
    ort_output = run_ort_inference(onnx_path, inputs, input_names)
    print ("Model output - \n{}".format(model_output))
    print ("ORT output - \n{}".format(ort_output))

In [19]:
def flatten(l):
    result = list()
    for subL in l:
        result.extend(subL)
    return result

In [20]:
def create_model_inputs(model_type, num_jets = 4, num_particles_per_jet = 1, device = 'cpu', expectedIntDtype = 'int32'):
    def get_feature_dimension(feature_name, model_types_dims_mapper, default_dims_mapper, model_type):
        dimensions = None
        if feature_name in default_dims_mapper.keys():
            dimensions = default_dims_mapper[feature_name]
        else:
            dimensions = model_types_dims_mapper[model_type][feature_name]
        return dimensions
    
    model_type_dims_mapper = {'ak8_points_pf_sv_full': {'pf_features': 25, 'sv_features': 11}, 'ak8_points_pf_sv_mass_decorr': {'pf_features': 20, 'sv_features': 11}, 'ak4_points_pf_sv_CHS_eta4p7': {'pf_features': 20, 'sv_features': 11}, 'ak8_points_pf_sv_mass_regression': {'pf_features': 25, 'sv_features': 11}}
    default_dims_mapper = {'pf_points': 2, 'pf_mask': 1, 'sv_points': 2, 'sv_mask': 1}
    input_names = ['pf_points', 'pf_features', 'pf_mask', 'sv_points', 'sv_features', 'sv_mask']
    input_names += ['batch_shapes_{}'.format(item) for item in input_names]
    
    inputs = list()
    for input_name in input_names:
        if 'batch_shapes_' not in input_name:
            dimensions = get_feature_dimension(input_name, model_type_dims_mapper, default_dims_mapper, model_type)    
            inputs.append(torch.rand((num_jets * num_particles_per_jet * dimensions, ), dtype = torch.float).to(device))
        else:
            dimensions = get_feature_dimension(input_name.replace('batch_shapes_', ''), model_type_dims_mapper, default_dims_mapper, model_type)    
            datatype = torch.int32 if expectedIntDtype == 'int32' else torch.int64
            inputs.append(torch.tensor(flatten([[dimensions, num_particles_per_jet] for _ in range(num_jets)]), dtype = datatype).to(device))
    return tuple(inputs), input_names

In [21]:
def get_model(model_type, device = 'cpu'):
    data_config_name = 'data/AK4/{}.yaml'.format(model_type)
    if model_type != 'ak4_points_pf_sv_CHS_eta4p7':
        data_config_name = 'data/{}.yaml'.format(model_type)
    import networks.particle_net_ak4_pf_sv as network_module1
    import networks.particle_net_pf_sv as network_module2
    import networks.particle_net_pf_sv_mass_regression as network_module3
    
    model_state_dict = data_config_name.replace('.yaml','.pt')
    jit_model_save = data_config_name.replace('.yaml','_ragged_gpu_jit.pt')
    onnx_model = data_config_name.replace('.yaml','.onnx')
    
    data_config = SimpleIterDataset([], data_config_name, for_training=False).config
    model, model_info = None, None
    
    network_module = None
    if model_type == 'ak8_points_pf_sv_full':
        network_module = network_module2
    elif model_type == 'ak8_points_pf_sv_mass_decorr':
        network_module = network_module2
    elif model_type == 'ak4_points_pf_sv_CHS_eta4p7':
        network_module = network_module1
    else:
        network_module = network_module3
    
    model, model_info = network_module.get_model(data_config, for_inference=True)
    model = torch.jit.script(model)
    model.load_state_dict(torch.load(model_state_dict, map_location=torch.device(device)))
    model.eval()
    
    return model, model_info

In [22]:
def export_multiple_models(model_types):
    models = list() 
    for model_type in model_types:
        print ("Export model type - {}".format(model_type)) 
        model, model_info = get_model(model_type)
        model_inputs, input_names = create_model_inputs(model_type, num_jets = 9, num_particles_per_jet = 3)
        onnx_path = "exported_models/{}.onnx".format(model_type)
        export_onnx(model, model_inputs, model_info['dynamic_axes'], input_names, model_info['output_names'], onnx_path)
        models.append(model)
    return models

In [23]:
def test_exported_models(models, model_types, configs = [{'num_jets': 1, 'num_particles_per_jet': 13}, {'num_jets': 1, 'num_particles_per_jet': 1}, {'num_jets': 17, 'num_particles_per_jet': 3}, {'num_jets': 16, 'num_particles_per_jet': 20}]):
    for index, model_type in enumerate(model_types):
        model = models[index]
        onnx_path = "exported_models/{}.onnx".format(model_type)
        print ("Model Type - {}\n".format(model_type))
        print ("------------")
        for config in configs:
            print ("Config - Num Jets = {} | Num particles per jet = {}".format(config['num_jets'], config['num_particles_per_jet']))
            test_model_inputs, test_model_input_names = create_model_inputs(model_type, **config)
            print ("PT inference - \n{}\n".format(run_pt_inference(model, test_model_inputs)))
            print ("ORT inference - \n{}\n".format(run_ort_inference(onnx_path, test_model_inputs, test_model_input_names)))

In [24]:
#Test PT model
model_type = 'ak8_points_pf_sv_full'
model_inputs, names = create_model_inputs(model_type)
run_pt_inference(get_model(model_type)[0], model_inputs)

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Source Location                                                              
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
        aten::mkldnn_convolution        41.75%      67.795ms        41.78%      67.844ms       7.538ms      36.09 Mb           0 b             9  ....8/lib/python/site-packages/torch/nn/modules/conv.py(443): _conv_forward  
                                                                                                        

tensor([[2.1865e-02, 6.8637e-07, 6.4621e-09, 7.1645e-13, 3.1260e-20, 1.1702e-16,
         7.4372e-07, 9.4940e-07, 9.3260e-11, 2.7153e-03, 5.4914e-06, 4.3287e-09,
         1.5496e-01, 5.7763e-04, 8.1777e-01, 2.1090e-03, 4.5155e-07, 1.3723e-08,
         3.6713e-15, 1.6167e-25],
        [9.9954e-01, 1.8797e-06, 1.1591e-07, 1.3732e-12, 5.3152e-11, 1.0017e-08,
         3.5293e-04, 3.0331e-05, 1.7735e-11, 1.6111e-06, 4.4681e-08, 3.7177e-12,
         1.1229e-05, 1.3669e-07, 4.6986e-06, 5.6337e-05, 9.6423e-10, 9.3342e-10,
         6.0830e-15, 7.1186e-26],
        [7.4018e-01, 1.1062e-03, 6.6063e-08, 1.3741e-09, 1.0844e-07, 5.9436e-05,
         1.1757e-03, 2.6706e-07, 9.2180e-11, 1.2576e-06, 1.2619e-07, 3.5501e-10,
         2.0316e-04, 7.0657e-06, 2.5725e-01, 1.5151e-05, 3.0153e-10, 6.7233e-09,
         1.9610e-17, 6.0341e-36],
        [3.2150e-05, 1.1261e-08, 2.8658e-08, 2.3015e-11, 3.3278e-24, 8.2558e-11,
         8.6748e-02, 6.7061e-08, 4.1312e-11, 6.1413e-06, 3.7265e-09, 3.0632e-11,
       

In [25]:
model_types = ['ak8_points_pf_sv_full', 'ak4_points_pf_sv_CHS_eta4p7', 'ak8_points_pf_sv_mass_decorr', 'ak8_points_pf_sv_mass_regression']
models = export_multiple_models(model_types)

Export model type - ak8_points_pf_sv_full


/Users/nirmalthomas/Library/Python/3.8/lib/python/site-packages/torch/onnx/symbolic_helper.py:719: UserWarning: allowzero=0 by default. In order to honor zero value in shape use allowzero=1
  warnings.warn("allowzero=0 by default. In order to honor zero value in shape use allowzero=1")
2022-11-07 18:16:10.644374 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_459'
2022-11-07 18:16:10.663616 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '184'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:10.663630 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '185'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:10.663637 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '174'. It is not u

Model output - 
tensor([[5.2472e-01, 1.1307e-01, 6.2028e-07, 8.6310e-08, 2.1301e-08, 5.4420e-05,
         1.5969e-02, 5.5414e-10, 7.2570e-10, 2.4125e-04, 6.0248e-11, 1.4272e-09,
         1.8701e-01, 4.4403e-09, 1.5819e-01, 7.4600e-04, 2.9107e-12, 1.6321e-06,
         3.5639e-18, 1.4696e-19],
        [1.2894e-01, 1.4772e-04, 2.6657e-07, 4.7445e-09, 1.4293e-12, 1.2452e-09,
         5.8327e-03, 3.1170e-05, 1.1815e-08, 7.6118e-03, 1.9609e-05, 4.8386e-08,
         1.0332e-01, 7.7149e-05, 7.5380e-01, 2.1889e-04, 6.5369e-07, 1.3122e-06,
         6.1282e-12, 7.7276e-21],
        [9.9038e-01, 9.4815e-06, 1.2748e-08, 2.0037e-13, 1.7461e-17, 2.2494e-09,
         3.0224e-03, 1.5983e-07, 2.8164e-13, 3.0545e-05, 1.5454e-09, 6.3218e-13,
         4.2614e-03, 1.9913e-07, 1.7270e-03, 5.7347e-04, 6.4928e-11, 1.2486e-10,
         1.7405e-20, 1.1171e-34],
        [2.6612e-03, 2.5503e-03, 1.0073e-03, 1.2958e-03, 2.6586e-07, 6.7891e-06,
         4.3576e-02, 1.1340e-04, 6.6539e-05, 3.1434e-01, 2.9731e-05, 1.4

2022-11-07 18:16:13.158064 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_459'
2022-11-07 18:16:13.174811 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '184'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:13.174826 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '185'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:13.174832 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '174'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:13.174838 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '181'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:13.174843 [W:onnxruntime:, gr

Model output - 
tensor([[1.8845e-01, 2.7349e-01, 9.4459e-02, 8.1806e-02, 6.8261e-03, 1.5388e-01,
         2.0109e-01, 1.4155e-06],
        [2.6895e-03, 1.1453e-05, 1.0236e-02, 3.5841e-04, 3.3342e-03, 7.9586e-02,
         4.6605e-02, 8.5718e-01],
        [3.3728e-01, 1.9112e-01, 1.6546e-01, 1.1601e-01, 1.2509e-02, 7.2156e-02,
         1.0539e-01, 7.6428e-05],
        [2.2686e-01, 4.3943e-01, 4.9339e-02, 3.6539e-02, 2.1000e-03, 6.3349e-02,
         1.8146e-01, 9.2299e-04],
        [1.6122e-01, 6.1216e-01, 4.8465e-02, 6.3720e-02, 2.9442e-04, 1.8954e-02,
         9.5180e-02, 1.0963e-05],
        [2.0513e-02, 9.7293e-01, 2.6028e-04, 2.3966e-03, 9.9060e-07, 8.4154e-05,
         3.8181e-03, 2.6906e-15],
        [1.8874e-01, 2.9103e-02, 6.7075e-02, 1.1245e-02, 2.4689e-02, 2.7710e-01,
         3.8886e-01, 1.3191e-02],
        [1.1531e-02, 9.8459e-01, 1.3611e-04, 1.9865e-03, 6.0612e-07, 3.7641e-05,
         1.7222e-03, 1.8771e-17],
        [3.9614e-02, 8.3492e-01, 7.0894e-03, 1.9568e-02, 1.9537e

2022-11-07 18:16:16.010414 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_459'
2022-11-07 18:16:16.028009 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '184'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:16.028024 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '185'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:16.028031 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '174'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:16.028037 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '181'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:16.028042 [W:onnxruntime:, gr

Model output - 
tensor([[1.5348e-11, 6.2136e-11, 5.3534e-27, 1.0000e+00, 1.8184e-07, 1.0113e-08,
         2.0744e-13, 1.9256e-40],
        [5.6951e-13, 2.3663e-09, 2.0404e-23, 1.0000e+00, 1.8224e-06, 1.5809e-07,
         4.0333e-14, 2.5726e-21],
        [2.4476e-08, 3.4888e-12, 4.5546e-25, 1.0000e+00, 2.1767e-22, 6.0520e-15,
         2.3351e-33, 4.3591e-36],
        [1.7418e-06, 7.8784e-07, 1.4541e-17, 1.0000e+00, 1.1049e-09, 1.6540e-10,
         1.1422e-14, 1.2090e-22],
        [4.3878e-13, 1.1314e-13, 1.5745e-31, 1.0000e+00, 8.4855e-15, 1.1397e-11,
         2.1173e-23, 3.2226e-36],
        [7.2572e-10, 8.5088e-11, 8.4095e-22, 1.0000e+00, 4.4529e-14, 1.9013e-10,
         3.6090e-22, 2.5641e-30],
        [8.3776e-22, 8.2122e-14, 1.8628e-33, 1.0000e+00, 6.1780e-16, 3.6019e-15,
         3.0829e-28, 0.0000e+00],
        [8.4134e-06, 1.0737e-07, 3.9016e-20, 9.9999e-01, 5.2902e-15, 3.6825e-06,
         1.6775e-20, 4.2842e-32],
        [2.0500e-08, 4.2122e-11, 5.6488e-24, 1.0000e+00, 3.4072e

2022-11-07 18:16:18.875966 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_459'
2022-11-07 18:16:18.891595 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '184'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:18.891608 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '185'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:18.891614 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '174'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:18.891619 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '181'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:18.891625 [W:onnxruntime:, gr

In [26]:
test_exported_models(models, model_types)

Model Type - ak8_points_pf_sv_full

------------
Config - Num Jets = 1 | Num particles per jet = 13
PT inference - 
tensor([[9.9717e-01, 1.5936e-03, 1.1684e-09, 8.0910e-12, 3.2486e-11, 3.3310e-10,
         2.1399e-04, 3.4275e-06, 9.1227e-10, 2.6549e-05, 2.1156e-09, 4.0993e-10,
         2.8850e-04, 1.2905e-08, 6.3273e-05, 6.4009e-04, 2.8466e-11, 4.3183e-10,
         6.4453e-18, 4.9030e-26]], grad_fn=<SoftmaxBackward0>)



2022-11-07 18:16:23.636800 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_459'
2022-11-07 18:16:23.652372 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '184'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:23.652384 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '185'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:23.652390 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '174'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:23.652396 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '181'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:23.652401 [W:onnxruntime:, gr

ORT inference - 
[array([[9.97170627e-01, 1.59361004e-03, 1.16836074e-09, 8.09101484e-12,
        3.24854310e-11, 3.33097744e-10, 2.13986146e-04, 3.42756266e-06,
        9.12273201e-10, 2.65493381e-05, 2.11565210e-09, 4.09929063e-10,
        2.88504263e-04, 1.29053035e-08, 6.32732481e-05, 6.40089274e-04,
        2.84663595e-11, 4.31832431e-10, 6.44536155e-18, 4.90299732e-26]],
      dtype=float32)]

Config - Num Jets = 1 | Num particles per jet = 1
PT inference - 
tensor([[9.9104e-01, 8.4595e-06, 6.0825e-06, 3.7309e-10, 1.8490e-10, 8.6133e-08,
         5.0306e-03, 4.8611e-05, 2.5785e-10, 1.6973e-04, 3.9118e-07, 4.4247e-10,
         2.3097e-03, 2.6691e-06, 3.9012e-04, 9.9179e-04, 5.3788e-08, 3.4778e-08,
         5.3975e-13, 2.6436e-21]], grad_fn=<SoftmaxBackward0>)



2022-11-07 18:16:24.540492 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_459'
2022-11-07 18:16:24.556350 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '184'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:24.556364 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '185'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:24.556371 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '174'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:24.556376 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '181'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:24.556381 [W:onnxruntime:, gr

ORT inference - 
[array([[9.9104160e-01, 8.4595085e-06, 6.0825419e-06, 3.7308778e-10,
        1.8490404e-10, 8.6134257e-08, 5.0305850e-03, 4.8610960e-05,
        2.5784935e-10, 1.6973270e-04, 3.9118717e-07, 4.4246914e-10,
        2.3097307e-03, 2.6691089e-06, 3.9012151e-04, 9.9179544e-04,
        5.3788774e-08, 3.4777830e-08, 5.3975270e-13, 2.6436014e-21]],
      dtype=float32)]

Config - Num Jets = 17 | Num particles per jet = 3
PT inference - 
tensor([[1.4427e-01, 5.2076e-02, 8.4423e-06, 9.4510e-07, 1.2903e-07, 1.5530e-04,
         2.8820e-02, 2.1219e-08, 1.3760e-08, 4.8182e-04, 7.0497e-09, 1.5444e-08,
         3.0165e-01, 1.5294e-06, 4.6212e-01, 1.0418e-02, 3.1009e-09, 1.1700e-06,
         5.2680e-16, 1.5087e-19],
        [8.6123e-01, 3.9115e-06, 8.4377e-05, 1.1744e-09, 1.4007e-11, 2.2377e-11,
         4.2257e-04, 5.6799e-05, 4.7638e-10, 6.3611e-02, 6.2121e-07, 2.2457e-10,
         5.0662e-02, 2.5061e-07, 2.3557e-02, 3.7059e-04, 9.1526e-13, 8.6515e-07,
         2.9421e-16, 6.5824e-2

...

ORT inference - 
[array([[1.]], dtype=float32)]

Config - Num Jets = 17 | Num particles per jet = 3
PT inference - 
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SoftmaxBackward0>)



2022-11-07 18:16:40.896691 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_459'
2022-11-07 18:16:40.913129 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '184'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:40.913143 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '185'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:40.913149 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '174'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:40.913154 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '181'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:40.913160 [W:onnxruntime:, gr

ORT inference - 
[array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)]

Config - Num Jets = 16 | Num particles per jet = 20
PT inference - 
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SoftmaxBackward0>)



2022-11-07 18:16:42.917292 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_459'
2022-11-07 18:16:42.936609 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '184'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:42.936624 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '185'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:42.936629 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '174'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:42.936634 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '181'. It is not used by any node and should be removed from the model.
2022-11-07 18:16:42.936639 [W:onnxruntime:, gr

ORT inference - 
[array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)]

